In [57]:
import os
from colorthief import ColorThief
import numpy as np
import cv2
import regex as re

In [33]:
VIDEO_PATH = '/home/leonamtv/Vídeos/BR2049.mp4'
# VIDEO_PATH = '/home/leonamtv/Vídeos/moonlight.mp4'
frame_pace = 600

In [39]:
os.path.isfile(VIDEO_PATH)

True

In [35]:
cap = cv2.VideoCapture(VIDEO_PATH)

In [36]:
if not cap.isOpened(): 
    print("could not open")
else:
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps    = int(cap.get(cv2.CAP_PROP_FPS))
    print(length, height, width, fps)

235152 800 1920 23


In [37]:
current_frame = 0
success, frame = cap.read()
while current_frame < length and success:
    cap.set(1, current_frame)
    if not cv2.imwrite('/home/leonamtv/MEGAsync/LINUX/PROJETOS/FRAME_ANALYSER/tmp/frame%d.png' % current_frame, frame):
        raise Exception("Could not write image")
    success, frame = cap.read()
    current_frame += frame_pace

In [76]:
storage_folder = '/home/leonamtv/MEGAsync/LINUX/PROJETOS/FRAME_ANALYSER/tmp'
output_folder  = '/home/leonamtv/MEGAsync/LINUX/PROJETOS/FRAME_ANALYSER/output'

reg = r"(?<=frame)([0-9]+)(?=\.png)"

html = "<html><body><title>Movie Palette</title><link rel='stylesheet' type='text/css' href='./style.css'><table>"
html += "<tr><th>Frame</th><th>1st MCC</th><th>2nd MCC</th><th>3rd MCC</th><th>4th MCC</th><th>5th MCC</th><th>6th MCC</th><th>7th MCC</th></tr>"

from functools import cmp_to_key

def sorting_function ( a, b ):
    an, bn = int(re.search(reg, a).group(0)), int(re.search(reg, b).group(0))
    return an - bn

for file in sorted(os.listdir(storage_folder), key=cmp_to_key(sorting_function)):
    colortf = ColorThief(os.path.join(storage_folder, file))
    dominant = colortf.get_palette(color_count=8)
    
    title = re.search(reg, file)
    
    html += "<tr><td class='title'>" + title.group(0) + '</td>'
    for color in dominant:
        style = "'background-color:rgb(%d, %d, %d)'" % color
        html += "<td style=" + style + "></td>"
    html += '</tr>'
html += '</table></body></html>'
file = open(os.path.join(output_folder, 'index.html'),'w')
file.write(html)
file.close()